In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
r = requests.get("https://spotifycharts.com/regional/us/daily/latest")

In [3]:
r.headers["Content-Type"]

'text/html; charset=UTF-8'

In [4]:
html = BeautifulSoup(r.text, 'html.parser')

In [5]:
title_artist = html.find_all('td', attrs = {'class':'chart-table-track'})
streams = html.find_all('td', attrs = {'class':'chart-table-streams'})
ids = html.find_all('td', attrs = {'class':'chart-table-image'})
ids = [i.a.get("href").replace("https://open.spotify.com/track/", "") for i in ids]

In [23]:
t = [{'title':i.strong.text,
      'artist':i.span.text.replace("by ", ""),
      'streams':int(j.text.replace(',', '')),
      'id':ids} for i, j, ids in zip(title_artist, streams, ids)]

More JSON-y

`{"title - artist":[{"streams":numberofstreams, "id":trackid}]}`

In [24]:
tj = {i.strong.text + " - " + i.span.text.replace("by ", ""):
      [{'streams':int(j.text.replace(',', '')),
      'id':ids}] for i, j, ids in zip(title_artist, streams, ids)}

Nice middle ground

In [26]:
list_kvpairs = [{i.strong.text + " - " + i.span.text.replace("by ", ""):
      [{'streams':int(j.text.replace(',', '')),
      'id':ids}]} for i, j, ids in zip(title_artist, streams, ids)]

In [27]:
list_kvpairs[:10]

[{'MIDDLE CHILD - J. Cole': [{'streams': 1366354,
    'id': '2JvzF1RMd7lE3KmFlsyZD8'}]},
 {'Sunflower - Spider-Man: Into the Spider-Verse - Post Malone': [{'streams': 1334168,
    'id': '3KkXRkHbMCARz0aVfEt68P'}]},
 {'7 rings - Ariana Grande': [{'streams': 1301804,
    'id': '6ocbgoVGwYJhOv1GgI9NsF'}]},
 {'Sucker - Jonas Brothers': [{'streams': 1255027,
    'id': '4y3OI86AEP6PQoDE6olYhO'}]},
 {'Murder On My Mind - YNW Melly': [{'streams': 1226383,
    'id': '7eBqSVxrzQZtK2mmgRG6lC'}]},
 {'Wow. - Post Malone': [{'streams': 1112766,
    'id': '6MWtB6iiXyIwun0YzU6DFP'}]},
 {'Going Bad (feat. Drake) - Meek Mill': [{'streams': 1075432,
    'id': '2IRZnDFmlqMuOrYOLnZZyc'}]},
 {"break up with your girlfriend, i'm bored - Ariana Grande": [{'streams': 1069252,
    'id': '4kV4N9D1iKVxx1KLvtTpjS'}]},
 {'Robbery - Juice WRLD': [{'streams': 1041054,
    'id': '6Hj9jySrnFppAI0sEMCZpJ'}]},
 {'Thotiana - Blueface': [{'streams': 965858,
    'id': '3oh6SCCeLuXhFpEyepla6G'}]}]

# "Stream" will look like this

In [25]:
from time import sleep
for song in t[:10]:
    print(song)
    sleep(1)

{'title': 'MIDDLE CHILD', 'artist': 'J. Cole', 'streams': 1366354, 'id': '2JvzF1RMd7lE3KmFlsyZD8'}
{'title': 'Sunflower - Spider-Man: Into the Spider-Verse', 'artist': 'Post Malone', 'streams': 1334168, 'id': '3KkXRkHbMCARz0aVfEt68P'}
{'title': '7 rings', 'artist': 'Ariana Grande', 'streams': 1301804, 'id': '6ocbgoVGwYJhOv1GgI9NsF'}
{'title': 'Sucker', 'artist': 'Jonas Brothers', 'streams': 1255027, 'id': '4y3OI86AEP6PQoDE6olYhO'}
{'title': 'Murder On My Mind', 'artist': 'YNW Melly', 'streams': 1226383, 'id': '7eBqSVxrzQZtK2mmgRG6lC'}
{'title': 'Wow.', 'artist': 'Post Malone', 'streams': 1112766, 'id': '6MWtB6iiXyIwun0YzU6DFP'}
{'title': 'Going Bad (feat. Drake)', 'artist': 'Meek Mill', 'streams': 1075432, 'id': '2IRZnDFmlqMuOrYOLnZZyc'}
{'title': "break up with your girlfriend, i'm bored", 'artist': 'Ariana Grande', 'streams': 1069252, 'id': '4kV4N9D1iKVxx1KLvtTpjS'}
{'title': 'Robbery', 'artist': 'Juice WRLD', 'streams': 1041054, 'id': '6Hj9jySrnFppAI0sEMCZpJ'}
{'title': 'Thotiana', 

# Scraping Function

In [ ]:
import requests
from bs4 import BeautifulSoup
from time import sleep

def get_chart(region = 'global', date = 'latest'):
    
    # create url to query
    url = "https://spotifycharts.com/regional/{}/daily/{}".format(region, date)
    
    # run query, if fails, re-try after 3 seconds
    r = requests.get(url)
    while r.status_code != 200:
        print("Error parsing {}\nError code: {}\nTrying again...\n".format(url, r.status_code))
        sleep(5)
        r = requests.get(url)
    
    # Parse HTML response for data
    html = BeautifulSoup(r.text, 'html.parser')
    title_artist = html.find_all('td', attrs = {'class':'chart-table-track'})
    streams = html.find_all('td', attrs = {'class':'chart-table-streams'})
    ids = html.find_all('td', attrs = {'class':'chart-table-image'})
    ids = [i.a.get("href").replace("https://open.spotify.com/track/", "") for i in ids]
    
    # Create a list of dictionaries with the data
    dta =[{'title':i.strong.text,
          'artist':i.span.text.replace("by ", ""),
          'streams':int(j.text.replace(',', '')),
          'id':ids,
          'date':date,
          'region':region} for i, j, ids in zip(title_artist, streams, ids)]
    
    return dta

In [ ]:
data = get_chart(region = 'fr', date = "2019-03-16")
for i in data[0:10]:
    print(i)
    sleep(1)

In [ ]:
with open("FR-2017-10-19.txt", "a+") as f:
    for song in data:
        f.write(str(song) + "\n")

#### The below cell doesn't really create valid JSON - it's 200 JSONs appended in a file...

In [ ]:
import json
with open("FR-2017-10-19.json", "a+") as f:
    for song in data:
        json.dump(song, f)

# Publish to a Kafka topic

In [ ]:
import kafka
import json

In [ ]:
data[0]

In [ ]:
producer = kafka.KafkaProducer(value_serializer=lambda m: json.dumps(m).encode('ascii'), bootstrap_servers=['localhost:9092'])
for record in data:
    producer.send('TutorialTopic', record)
    sleep(1)

In [ ]:
# need to pass a JSON desserializer
consumer = kafka.KafkaConsumer('TutorialTopic', bootstrap_servers=['localhost:9092'], auto_offset_reset='earliest', enable_auto_commit=False)

In [ ]:
for message in consumer:
    print("%s:%d:%d: key=%s value=%s" % (message.topic, message.partition,
                                         message.offset, message.key,
                                         message.value))